In [1]:
# import libraries
import ee
import time

In [2]:
# Authenticate and initialize the Earth Engine
# service_account = 'edsmlice@edsml-irp.iam.gserviceaccount.com'
# credentials = ee.ServiceAccountCredentials(service_account, 'edsml-irp-2696518a7f6f.json')
try:
    ee.Authenticate()
    ee.Initialize()
except Exception as e:
    print("The Earth Engine was not initialized. Please check your credentials.")
    raise e


In [3]:
# Define a method to get the image
def get_images(lat1, lon1, lat2, lon2, start_date="2014-01-01", end_date="2024-05-31"):
    rectangle = ee.Geometry.Rectangle([lon1, lat1, lon2, lat2])
    
    # Retrieve the image collection, filtering by bounds and cloud coverage
    collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
                  .filterBounds(rectangle) \
                  .filterDate(start_date, end_date) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))

    # This function processes each image in the collection
    def process_image(image):
        # Select bands for Red, Green, Blue, and NIR
        selected_bands = image.select(['B4', 'B3', 'B2', 'B8']).toFloat()

        # Calculate NDVI
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI').toFloat()

        # Calculate NDWI
        ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI').toFloat()
        
        # Combine all bands into one image
        combined_image = selected_bands.addBands([ndvi, ndwi])

        # Clip the image to the defined rectangular area
        clipped_image = combined_image.clip(rectangle)

        return clipped_image

    # Apply the process to each image in the collection
    processed_collection = collection.map(process_image)
    
    return processed_collection

In [6]:
def export_images(collection, region, folder_name, area_name, scale=10, batch_size=20):
    count = collection.size().getInfo()  # Retrieve the number of images to export
    print(f"Total images to export: {count}")

    images_list = collection.toList(count)
    
    for i in range(0, count, batch_size):  # Process in batches
        end = i + batch_size if i + batch_size < count else count
        for j in range(i, end):
            image = ee.Image(images_list.get(j))
            date = image.date().format('YYYY-MM-dd').getInfo()
            description = f"{folder_name}_{area_name}_{date}_{j}"
            file_name_prefix = f"{area_name}_{date}_{j}"
            full_folder_name = f"Images_for_detection/{folder_name}"

            export_params = {
                'description': description,
                'scale': scale,
                'region': region,
                'folder': full_folder_name,
                'fileNamePrefix': file_name_prefix,
                'fileFormat': 'GeoTIFF'
            }

            task = ee.batch.Export.image.toDrive(image, **export_params)
            task.start()
            print(f"Started export task {description} in folder {full_folder_name}.")
        
        time.sleep(10)  # Wait for 10 seconds before starting the next batch


In [21]:
# Define the location and date range
lat1, lon1 = 5.281720, 73.604446
lat2, lon2 = 5.278788, 73.610352

# Get the image
images = get_images(lat1, lon1, lat2, lon2)

# Define the location, date range, and area name
area_name = "Olhuvelifushi"

# Define the export region, folder name for subfolder, and area name
export_region = ee.Geometry.Rectangle([lon1, lat1, lon2, lat2]).getInfo()['coordinates']
export_folder_name = 'Olhuvelifushi'

# Call the export function
export_images(images, export_region, export_folder_name, area_name)

Total images to export: 146
Started export task Olhuvelifushi_Olhuvelifushi_2015-11-24_0 in folder Images_for_detection/Olhuvelifushi.
Started export task Olhuvelifushi_Olhuvelifushi_2015-12-24_1 in folder Images_for_detection/Olhuvelifushi.
Started export task Olhuvelifushi_Olhuvelifushi_2015-12-24_2 in folder Images_for_detection/Olhuvelifushi.
Started export task Olhuvelifushi_Olhuvelifushi_2016-02-12_3 in folder Images_for_detection/Olhuvelifushi.
Started export task Olhuvelifushi_Olhuvelifushi_2016-02-12_4 in folder Images_for_detection/Olhuvelifushi.
Started export task Olhuvelifushi_Olhuvelifushi_2016-03-23_5 in folder Images_for_detection/Olhuvelifushi.
Started export task Olhuvelifushi_Olhuvelifushi_2016-03-23_6 in folder Images_for_detection/Olhuvelifushi.
Started export task Olhuvelifushi_Olhuvelifushi_2016-07-11_7 in folder Images_for_detection/Olhuvelifushi.
Started export task Olhuvelifushi_Olhuvelifushi_2016-07-11_8 in folder Images_for_detection/Olhuvelifushi.
Started e